In [1]:
import pandas as pd
import numpy as np
import nltk
import pyspark
import matplotlib.pyplot as plt
import os.path as osp
import os
import shutil
from open_secrets_schema import Schema

%matplotlib notebook

In [2]:
os.chdir('/home/matt/etl/open-secrets/Expend18/')

In [10]:
# !unzip -q Expend18.zip -d Expend18/

In [15]:
# !ls Expend18/

Expend18_Read_Me.txt  expends18.txt


In [16]:
# !mv Expend18/* . && rm -rf Expend18/

In [3]:
!ls

Expend18_Read_Me.txt  expends18.sql  pq_crp_expends18.sql
Expend18.zip	      expends18.txt


In [4]:
!head -n 10 expends18.txt

|2018|,1,|SB21B-11865-10000|,|C00460147|,|DP|,|DNC/Travel Offset Account|,|W Los Angeles|,|F30|,554,01/18/2017,|Los Angeles|,|CA|,|90024    |,|         |,||,,|Lodging & Catering|,|     |,||,|ORG|,|@new|
|2018|,2,|SB21B-11866-10000|,|C00460147|,|DP|,|DNC/Travel Offset Account|,|W Los Angeles|,|F30|,5064,01/18/2017,|Los Angeles|,|CA|,|90024    |,|         |,||,,|Lodging & Catering|,|     |,||,|ORG|,|@new|
|2018|,3,|SB21B-11867-10000|,|C00460147|,|DP|,|DNC/Travel Offset Account|,|Budget Rent A Car|,|A40|,229,01/18/2017,|Avoca|,|PA|,|18641    |,|         |,||,,|Car Rental|,|     |,||,|ORG|,|@new|
|2018|,4,|SB21B-11867-20000|,|C00460147|,|DP|,|DNC/Travel Offset Account|,|Budget Rent A Car                                 |,|A40|,1519,01/18/2017,|Beverly Hills|,|CA|,|90210    |,|         |,||,,|Car Rental|,|     |,||,|ORG|,|@new|
|2018|,5,|SB21B-11867-30000|,|C00460147|,|DP|,|DNC/Travel Offset Account|,|BUDGET-RENT-A-CAR|,|A40|,50,01/18/2017,|MANCHESTER|,|NH|,|03101    |,|         |,||,,|Car 

In [19]:
def etl_text_file(text_file_path, schema_str, table_name, data_source_name, column_datetime_formats=None):
    schema = Schema(schema_str)
    schema.register_column_datetime_formats(column_datetime_formats)
    textFileName = osp.split(text_file_path)[1]
    stagingFolder = f'/staging/{table_name}'
    !hdfs dfs -mkdir -p {stagingFolder}
    !hdfs dfs -put {text_file_path} {stagingFolder}/{textFileName}
    with open(f'{table_name}.sql', 'w') as f:
        f.write(schema.hive_ddl(tableName=table_name, externalTableLocation=stagingFolder))
    !/opt/anaconda3/bin/beeline -u jdbc:hive2://cap1:10000 -n matt -p anything -f {table_name}.sql
    
    # Convert to the proper schema and write as parquet
    finalTableName = f'pq_{data_source_name}_{table_name}'
    with open(f'{finalTableName}.sql', 'w') as f:
        f.write(schema.hive_ddl_parquet(tableName=finalTableName, originalTableName=table_name))
    !/opt/anaconda3/bin/beeline -u jdbc:hive2://cap1:10000 -n matt -p anything -f {finalTableName}.sql
    
    # clean up -- drop the intermediate (text file) table and delete staged data from HDFS
    dropTableCmd = f'DROP TABLE {table_name};'
    !/opt/anaconda3/bin/beeline -u jdbc:hive2://cap1:10000 -n matt -p anything -e 'DROP TABLE {table_name};'
    !hdfs dfs -rm -r -f {stagingFolder}
    
    print('done')

`expends18` schema str: https://www.opensecrets.org/resources/datadictionary/Data%20Dictionary%20Expenditures.htm

In [6]:
expends18SchemaStr = ['''Cycle
Last year (even) of a federal 2-yr election cycle
Text (4)
CRP''',

'''ID
An auto ID added when dbo_Expenditures is made- acts as a unique identifier. This field cannot be used to match records from one download to the next.
Integer
CRP''',

'''TransID
A unique record identifier within a given cycle.
Text (20)
FEC''',

'''CRPFilerid
ID of the filing committee, same as Filerid unless it is a candidate committee, in which case it will be the candidate's unique id. (Note that a candidate can have more than one committee - this field indicates the exact committee receiving the contribution)
Text (9)
CRP''',

'''Recipcode
A two character code defining the type of recipient. For candidates, the first character is party ("D" for Democratic, "R" for Republican, "3" for Independent, Libertarian or third party, "U" for Unknown.) The second character is "W" for Winner, "L" for Loser, "I" for incumbent, "C" for Challenger, "O" for "Open Seat", and "N" for Non-incumbent. "N" is reserved for candidates that are neither in office nor running during the cycle in question. For party committees, the first character is party and the second character is "P." For PACs, the first character is "P" and for outside spending groups, the first character is "O". For both, the second character is "B" for Business, "L" for Labor", "I" for Ideological, "O" for "Other" and "U" for unknown.
Text (2)
CRP''',

'''Pacshort
The standardized name of the filing committee
Text (50)
CRP''',

'''CRPRecipname
The recipient of the disbursement. This field is standardized when possible
Text (90)
CRP''',

'''Expcode
The expenditure code assigned by the researcher.
Text (3)
CRP''',

'''Amount
The amount spent. This will be negative for refunds.
Float
FEC''',

'''Date
The reported date of the expenditure
mm/dd/yyyy
FEC''',

'''City
City of the recipient
Text (30)
FEC''',

'''State
State of the recipient
Text (2)
FEC''',

'''Zip
Zip of the recipient
Text (5)
FEC''',

'''CmteID_EF
Alternate committee ID
Text (9)
FEC''',

'''Candid
candidate id
Text (9)
FEC''',

'''Type
The transaction type code for the contribution. 15 is a contribution, 15e is an earmarked contribution (made through a group such as Club for Growth or Emily's List), 15j is a contribution through a joint fund raising committee and 22y is a refund. "10" indicates "soft" or nonfederal money for the 2002 cycle and earlier. For the 2004 cycle and later type "10" indicates Levin funds. Mostly empty
Text (3)
FEC''',

'''Descrip
Description of the expenditure
Text (100)
FEC''',

'''PG
Indicates whether expenditure is for the primary or general election, plus the year. Can also be 'S' for special elections, 'R' for runoff or 'O' for other. Not always filled in
Text (5)
FEC''',

'''ElecOther
Explanation if there is anything other than 'P' or 'G' in previous field
Text (20)
FEC''',

'''EntType
A filer-provided code for classifying the recipient. CCM = Candidate Cmte, PTY = Party Cmte, ORG & IND. Since these are applied differently across filing committees, these codes can be helpful in coding but are not reliable
Text (3)
FEC''',

'''Source
The source researcher used to assign a certain code to a disbursement. A list can be found in the Expenditures methodology
Text (5)
CRP''']

In [7]:
etl_text_file('expends18.txt', expends18SchemaStr, 'expends18', 'crp', column_datetime_formats={'Date': 'MM/dd/yyyy'})

2018-10-14 19:28:09,606 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2018-10-14 19:28:11,685 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
put: `/staging/expends18/expends18.txt': File exists
Connecting to jdbc:hive2://cap1:10000
2018-10-14 19:28:13 INFO  Utils:310 - Supplied authorities: cap1:10000
2018-10-14 19:28:13 INFO  Utils:397 - Resolved authority: cap1:10000
2018-10-14 19:28:13 INFO  HiveConnection:203 - Will try to open client transport with JDBC Uri: jdbc:hive2://cap1:10000
Connected to: Spark SQL (version 2.3.2)
Driver: Hive JDBC (version 1.2.1.spark2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://cap1:10000> 
0: jdbc:hive2://cap1:10000> DROP TABLE IF EXISTS expends18;
+---------+--+
| Result  |
+---------+--+
+---------+--+
No rows selected (0.113 seconds)
0: jdbc:hive2://cap1:10000>   

In [18]:
!/opt/anaconda3/bin/beeline -u jdbc:hive2://cap1:10000 -n matt -p anything -e 'SHOW TABLES;'

Connecting to jdbc:hive2://cap1:10000
2018-10-14 19:34:17 INFO  Utils:310 - Supplied authorities: cap1:10000
2018-10-14 19:34:17 INFO  Utils:397 - Resolved authority: cap1:10000
2018-10-14 19:34:17 INFO  HiveConnection:203 - Will try to open client transport with JDBC Uri: jdbc:hive2://cap1:10000
Connected to: Spark SQL (version 2.3.2)
Driver: Hive JDBC (version 1.2.1.spark2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
+-----------+-----------------------+--------------+--+
| database  |       tableName       | isTemporary  |
+-----------+-----------------------+--------------+--+
| default   | pac_spending          | false        |
| default   | pac_spending_by_type  | false        |
| default   | pq_crp_cands18        | false        |
| default   | pq_crp_cmtes18        | false        |
| default   | pq_crp_expends18      | false        |
| default   | pq_crp_indivs18       | false        |
| default   | pq_crp_pac_other18    | false        |
| default   | pq_crp_pacs18       

In [20]:
!/opt/anaconda3/bin/beeline -u jdbc:hive2://cap1:10000 -n matt -p anything -e 'SELECT * FROM pq_crp_expends18 LIMIT 10;'

Connecting to jdbc:hive2://cap1:10000
2018-10-14 19:35:06 INFO  Utils:310 - Supplied authorities: cap1:10000
2018-10-14 19:35:06 INFO  Utils:397 - Resolved authority: cap1:10000
2018-10-14 19:35:06 INFO  HiveConnection:203 - Will try to open client transport with JDBC Uri: jdbc:hive2://cap1:10000
Connected to: Spark SQL (version 2.3.2)
Driver: Hive JDBC (version 1.2.1.spark2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
+--------+----------+--------------+-------------+------------+-------------+----------------------------+----------+---------+-------------+------------+--------+------------+------------+---------+-------+-----------------------+--------+------------+----------+---------+--+
| Cycle  |    ID    |   TransID    | CRPFilerid  | Recipcode  |  Pacshort   |        CRPRecipname        | Expcode  | Amount  |    Date     |    City    | State  |    Zip     | CmteID_EF  | Candid  | Type  |        Descrip        |   PG   | ElecOther  | EntType  | Source  |
+--------+-------